In [1]:
#!/usr/bin/env python3
"""
Multi-agent Double DQN sur l'environnement Highway-fast-v0 avec 2 véhicules contrôlés.
Affiche les courbes de reward, reward moyen, et évolution d'epsilon et du learning rate pour chaque agent.
"""

import gymnasium as gym
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import highway_env

# --------------------------------------------------------------------------
# Chargement de la configuration de base
# --------------------------------------------------------------------------
with open("../config/1-highway-discrete-config.pkl", "rb") as f:
    base_config = pickle.load(f)

# --------------------------------------------------------------------------
# Création et configuration de l'environnement multi-agent
# --------------------------------------------------------------------------
env = gym.make("highway-fast-v0", render_mode="human")
env.unwrapped.config.update(base_config)
env.unwrapped.config.update({
    "controlled_vehicles": 2,
    "vehicles_count": 50,  # exemple : nombre total de véhicules
    "action": {
        "type": "MultiAgentAction",
        "action_config": {"type": "DiscreteMetaAction"}
    },
    "observation": {
        "type": "MultiAgentObservation",
        "observation_config": {"type": "Kinematics"}
    },
    "scaling": 7.5,
    "centering_position": [0.5, 0.5]
})
obs, info = env.reset(seed=0)

# --------------------------------------------------------------------------
# Fonctions d'aide
# --------------------------------------------------------------------------
def is_risky(state):
    grid_range = base_config["observation"]["grid_size"][0]
    grid_step  = base_config["observation"]["grid_step"][0]
    nx = int((grid_range[1] - grid_range[0]) / grid_step)
    ny = int((base_config["observation"]["grid_size"][1][1] - base_config["observation"]["grid_size"][1][0]) / base_config["observation"]["grid_step"][1])
    num_features = 7
    if state.size != nx * ny * num_features:
        return False
    grid = state.reshape(nx, ny, num_features)
    return np.any(grid[-1, :, 0] > 0.5)

def shape_reward(reward, action, state):
    shaped = reward
    # renforcement des penalités de collision
    if reward <= base_config.get("collision_reward", -1):
        shaped = reward * 2
    # pénalité pour action risquée
    if action == 1 and is_risky(state):
        shaped -= 0.5
    return shaped

# --------------------------------------------------------------------------
# Réseau de neurones et agent DQN (Double DQN + masquage)
# --------------------------------------------------------------------------
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class DQNAgent:
    def __init__(self, state_dim, action_dim):
        self.q_net     = DQN(state_dim, action_dim)
        self.target_net= DQN(state_dim, action_dim)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=1e-3)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=100, gamma=0.9)
        self.replay_buffer = deque(maxlen=100000)
        self.gamma     = 0.99
        self.epsilon   = 1.0
        self.epsilon_min   = 0.05
        self.epsilon_decay = 0.995
        self.batch_size    = 64
        self.action_dim    = action_dim

    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            acts = list(range(self.action_dim))
            if is_risky(state) and 1 in acts:
                acts.remove(1)
            return random.choice(acts) if acts else 1
        state_t = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_vals = self.q_net(state_t).squeeze(0).numpy()
        if is_risky(state):
            q_vals[1] = -np.inf
        return int(np.argmax(q_vals))

    def update_epsilon(self, ep, total_ep):
        if ep > total_ep // 2:
            self.epsilon = self.epsilon_min
        else:
            self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)

    def update(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        batch = random.sample(self.replay_buffer, self.batch_size)
        s, a, r, s2, d = map(np.array, zip(*batch))
        s  = torch.FloatTensor(s)
        a  = torch.LongTensor(a).unsqueeze(1)
        r  = torch.FloatTensor(r).unsqueeze(1)
        s2 = torch.FloatTensor(s2)
        d  = torch.FloatTensor(d).unsqueeze(1)

        q      = self.q_net(s).gather(1, a)
        with torch.no_grad():
            next_a = self.q_net(s2).argmax(dim=1, keepdim=True)
            q_next = self.target_net(s2).gather(1, next_a)
            target = r + self.gamma * (1 - d) * q_next

        loss = F.mse_loss(q, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# --------------------------------------------------------------------------
# Entraînement multi-agent
# --------------------------------------------------------------------------
def train_multi_agents(agents, env, num_episodes=100, sync_target=10, collision_threshold=0.5):
    rewards_hist = {i: [] for i in range(len(agents))}
    eps_hist     = {i: [] for i in range(len(agents))}
    lr_hist      = {i: [] for i in range(len(agents))}

    for ep in range(num_episodes):
        obs, info = env.reset()
        states = [o.flatten() for o in obs]
        ep_rewards = [0.0 for _ in agents]
        done = False
        step = 0

        while not done:
            actions = tuple(agent.get_action(s) for agent, s in zip(agents, states))
            next_obs, raw_reward, term, trunc, infos = env.step(actions)

            # --- Récupération du reward par agent ---
            if isinstance(raw_reward, (list, tuple, np.ndarray)):
                rewards = list(raw_reward)
            elif "agents_rewards" in infos:
                rewards = list(infos["agents_rewards"])
            else:
                rewards = [
                    env.unwrapped._agent_reward(actions[i], env.unwrapped.controlled_vehicles[i])
                    for i in range(len(agents))
                ]

            # --- Collision inter-agent ---
            pos = [veh.position for veh in env.unwrapped.controlled_vehicles]
            dist = np.linalg.norm(np.array(pos[0]) - np.array(pos[1]))
            if dist < collision_threshold:
                print(f"[Ep{ep} | Step{step}] Collision entre agents détectée (dist={dist:.2f})")

            next_states = [no.flatten() for no in next_obs]
            done = term or trunc

            # --- Stockage et apprentissage séparé ---
            for i, agent in enumerate(agents):
                sr = shape_reward(rewards[i], actions[i], states[i])
                agent.replay_buffer.append((states[i], actions[i], sr, next_states[i], float(done)))
                agent.update()
                ep_rewards[i] += rewards[i]

            states = next_states
            step += 1

        # --- Fin d'épisode : epsilon, scheduler, historiques ---
        for i, agent in enumerate(agents):
            agent.update_epsilon(ep, num_episodes)
            agent.scheduler.step()
            rewards_hist[i].append(ep_rewards[i])
            eps_hist[i].append(agent.epsilon)
            lr_hist[i].append(agent.optimizer.param_groups[0]['lr'])

        if ep % sync_target == 0:
            for agent in agents:
                agent.target_net.load_state_dict(agent.q_net.state_dict())

        if ep % 50 == 0:
            epsilons = [f"{a.epsilon:.3f}" for a in agents]
            print(f"[Episode {ep}] Réwards: {ep_rewards}, Epsilons: {epsilons}")

    return rewards_hist, eps_hist, lr_hist

# --------------------------------------------------------------------------
# Visualisation
# --------------------------------------------------------------------------
def plot_agent_metrics(hist, label, window=20):
    plt.figure()
    for i, vals in hist.items():
        plt.plot(vals, label=f"Agent {i+1}")
    plt.xlabel("Episode")
    plt.ylabel(label)
    plt.legend()
    plt.title(f"Évolution de {label}")
    plt.show()

# --------------------------------------------------------------------------
# Exécution principale
# --------------------------------------------------------------------------
if __name__ == "__main__":
    # Dimensions d'état et d'action
    state_dim = obs[0].flatten().shape[0]
    action_spaces = env.action_space.spaces if hasattr(env.action_space, 'spaces') else env.action_space
    action_dims = [space.n for space in action_spaces]

    # Création des agents
    agents = [DQNAgent(state_dim, ad) for ad in action_dims]

    # Entraînement
    rewards_hist, eps_hist, lr_hist = train_multi_agents(agents, env, num_episodes=200, sync_target=10)

    # Affichage des courbes
    plot_agent_metrics(rewards_hist, "Reward par épisode")
    # Reward moyen mobile
    plt.figure()
    window = 20
    for i, r in rewards_hist.items():
        mov = np.convolve(r, np.ones(window)/window, mode='valid')
        plt.plot(range(window-1, len(r)), mov, label=f"Agent {i+1}")
    plt.xlabel("Episode")
    plt.ylabel(f"Reward moyen (fenêtre {window})")
    plt.legend()
    plt.title("Reward moyen sur 20 épisodes")
    plt.show()

    plot_agent_metrics(eps_hist, "Epsilon")
    plot_agent_metrics(lr_hist, "Learning Rate")


FileNotFoundError: [Errno 2] No such file or directory: '../config/1-highway-discrete-config.pkl'